In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('marketing_campaign_Dataset.csv')
pd.set_option('display.max_columns', None)

df.head()

,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Age_Group,Gen_Group,DaysAsCustomer,NumChildren,NumAcceptedCmp,Total_Purchases,Total_Spending,Conversion_Rate
0,Master,Single,491796.22,1,0,2013-12-26,31,1364,72,1129,45,69,206,0,1,21,6,14,1,1,0,0,0,0,1,44,Late Adult,Gen X,3200,1,3,28,2885,0.2
1,Master,Married,513926.78,1,0,2014-05-04,23,1275,94,455,217,104,61,0,8,14,9,5,1,1,0,0,0,0,1,48,Mature,Gen X,3071,1,3,31,2206,0.6
2,Graduation,Alone,521309.22,2,2,2013-02-15,93,62,127,530,252,237,170,12,26,18,1,1,1,0,0,0,1,0,1,50,Mature,Gen X,3514,4,3,57,1378,3.0
3,PhD,YOLO,231628.91,1,0,2013-06-12,13,251,87,1153,157,241,197,4,26,18,3,14,0,1,1,0,1,0,0,111,Late Senior,Boomers,3397,1,3,51,2086,0.2
4,2n Cycle,Divorced,223936.10,1,1,2013-05-30,65,670,10,650,34,96,321,6,2,23,4,13,1,0,0,1,1,0,0,91,Late Senior,Boomers,3410,2,3,35,1781,0.2


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100279 entries, 0 to 100278
Data columns (total 34 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Education            100279 non-null  object 
 1   Marital_Status       100279 non-null  object 
 2   Income               100279 non-null  float64
 3   Kidhome              100279 non-null  int64  
 4   Teenhome             100279 non-null  int64  
 5   Dt_Customer          100279 non-null  object 
 6   Recency              100279 non-null  int64  
 7   MntWines             100279 non-null  int64  
 8   MntFruits            100279 non-null  int64  
 9   MntMeatProducts      100279 non-null  int64  
 10  MntFishProducts      100279 non-null  int64  
 11  MntSweetProducts     100279 non-null  int64  
 12  MntGoldProds         100279 non-null  int64  
 13  NumDealsPurchases    100279 non-null  int64  
 14  NumWebPurchases      100279 non-null  int64  
 15  NumCatalogPurchas

In [4]:
df.loc[(df.ID.duplicated())==True]

AttributeError: 'DataFrame' object has no attribute 'ID'

In [ ]:
df.isnull().sum()

In [ ]:

df['Dt_Data_Collected'] = pd.to_datetime('01-01-2025', dayfirst=True)
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], dayfirst=True, errors='coerce')


df['Age'] = df['Dt_Data_Collected'].dt.year - df['Year_Birth']

bins_age = [25, 34, 44, 54, 64, max(df['Age'])]
group_labels_age = ['Adult', 'Late Adult', 'Mature', 'Senior', 'Late Senior']
df['Age_Group'] = pd.cut(df['Age'], bins=bins_age, labels=group_labels_age)


bins_gen = [1893, 1964, 1980, 1996]
group_labels_gen = ['Boomers', 'Gen X', 'Millennials']
df['Gen_Group'] = pd.cut(df['Year_Birth'], bins=bins_gen, labels=group_labels_gen)


df['DaysAsCustomer'] = (df['Dt_Data_Collected'] - df['Dt_Customer']).dt.days


df['NumChildren'] = df['Kidhome'] + df['Teenhome']
df['NumAcceptedCmp'] = (df['AcceptedCmp1'] + df['AcceptedCmp2'] + 
                        df['AcceptedCmp3'] + df['AcceptedCmp4'] + 
                        df['AcceptedCmp5'] + df['Response'])
df['Total_Purchases'] = (df['NumDealsPurchases'] + df['NumWebPurchases'] + 
                         df['NumCatalogPurchases'] + df['NumStorePurchases'])
df['Total_Spending'] = (df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] + 
                        df['MntFishProducts'] + df['MntSweetProducts'] + df['MntGoldProds'])

# Conversion Rate (avoiding division errors)
df['Conversion_Rate'] = df['NumAcceptedCmp'] / df['NumWebVisitsMonth']
df['Conversion_Rate'].fillna(0, inplace=True)  # Replace NaN if any division by zero occurs
df['Conversion_Rate'] = df['Conversion_Rate'].round(1)


In [ ]:
df.sample(5)

In [ ]:
df.drop(['ID', 'Year_Birth', 'Z_CostContact', 'Z_Revenue', 'Dt_Data_Collected'], axis=1, inplace=True)


In [ ]:
df.to_csv('marketing_campaign_Dataset.csv', index=False)

In [ ]:
num_cols = df.select_dtypes(include = 'number').columns
cat_cols = df.select_dtypes(include = ['object','category']).columns

df[cat_cols].describe().T

In [ ]:
df[num_cols].describe().T

In [ ]:
num_cols = ['Income', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'Age', 'DaysAsCustomer', 'Total_Purchases', 'Total_Spending',
       'Conversion_Rate']
len(num_cols)

In [ ]:
plt.figure(figsize=(15, 20))
for i in range(len(num_cols)):
    plt.subplot(5, 4, i+1)
    sns.kdeplot(df[num_cols[i]])
    plt.tight_layout()

In [ ]:
plt.figure(figsize=(15, 25))
for i in range(len(num_cols)):
    plt.subplot(5, 4, i+1)
    sns.boxplot(y=df[num_cols[i]])
    plt.tight_layout()

In [ ]:
cols = ['Kidhome', 'Teenhome', 
        'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Response', 
       'NumChildren', 'NumAcceptedCmp']
cols.extend(cat_cols)
len(cols)

In [ ]:
plt.figure(figsize=(22, 20))
for i in range(len(cols)):
    plt.subplot(4, 4, i+1)
    sns.countplot(df[cols[i]])
    plt.tight_layout()

In [ ]:
plt.figure(figsize=(16, 12))
num_cols = ['Income', 'Recency', 'NumWebVisitsMonth',
       'Age', 'DaysAsCustomer', 'Total_Purchases', 'Total_Spending',
       'Conversion_Rate', 'NumChildren', 'NumAcceptedCmp']
sns.heatmap(df[num_cols].corr(), cmap='Blues', annot=True, fmt='.2f', linewidth=1)
plt.tight_layout()
plt.show()